In [1]:
import numpy as np


#加载数据
def load_data(file_name):
    #可分不可分都可以处理
    with open(file_name) as fr:
        lines = fr.readlines()

    x = np.ones((len(lines), 5), dtype=float)
    y = np.empty(len(lines), dtype=float)

    for i in range(len(lines)):
        line = lines[i].strip().split(',')
        x[i] = line[:5]
        y[i] = line[5]

    y = y.astype(np.int)
    y[y == 0] = -1

    x -= x.mean(axis=0)
    x /= x.std(axis=0)

    return x, y


x, y = load_data('features.csv')
x[:5], y[:5]

(array([[-0.96838565, -0.90569186,  0.34212957, -0.28760639, -1.05929159],
        [-1.29329276, -1.90829615,  0.34212957, -0.28760639, -0.66126443],
        [-0.48102498,  0.31923046,  0.34212957, -0.28760639,  0.43599101],
        [ 0.00633569,  0.65053435,  0.34212957, -0.28760639,  0.58018937],
        [ 2.28068547,  1.50120112, -0.69398406,  2.9859242 , -1.05319281]]),
 array([ 1,  1, -1, -1,  1]))

In [2]:
#常量
N, M = x.shape

w = np.empty(M)
w.fill(1 / M)
b = 0.0

In [3]:
def predict(x):
    return w.dot(x) + b


predict(x[0])

-0.5757691838088779

In [4]:
#现在的问题就是要max 最小几何距离 对每一个xy满足 几何距离 >= 最小几何距离
#写成函数就是 max min_geometric_marginn st. geometric_marginn(x,y) >= min_geometric_marginn

#试一下用梯度下降的方法来找到这个解

#合页损失函数
def get_loss():
    #loss分两部分,第1部分是w越大loss越大,第2部分是预测错误的loss
    lam = 0.2  #平衡两部分loss的变量
    l2_w = (w**2).sum()  #w越大loss越大

    loss = 0
    for i in range(N):
        pred = y[i] * (w.dot(x[i]) + b)
        pred = 1 - pred  #如果分类正确,那至少要正确到1以上,即使计算结果是0.9,也认为是错误
        relu = pred if pred > 0 else 0  #本来正数是预测正确,上面因为用1减了,所以变号了,现在是负数才是预测正确
        loss += relu
    return loss + lam * l2_w


get_loss()

1016.9036562947388

In [5]:
#我发明的暴力求梯度的方法.
def get_gradient():
    global w
    global b

    #这个变量就是增量了
    upsilon = 1e-5

    #先算出给增量前的loss
    loss = get_loss()

    #求w梯度
    grad_w = np.empty(M)
    for i in range(M):
        w[i] += upsilon
        loss2 = get_loss()
        w[i] -= upsilon
        grad_w[i] = (loss2 - loss) / upsilon

    #求b梯度
    b += upsilon
    loss2 = get_loss()
    b -= upsilon
    grad_b = (loss2 - loss) / upsilon

    return grad_w, grad_b


get_gradient()

(array([-460.36406032, -420.69229776,  528.26247623,  -67.84050144,
         665.35267489]),
 -7.9999999911706245)

In [6]:
for epoch in range(300):
    grad_w, grad_b = get_gradient()
    w -= grad_w * 1e-3
    b -= grad_b * 1e-3

    if epoch % 100 == 0:
        print(epoch, get_loss())

0 415.8723941113253
100 231.2302407539765
200 230.39371165862832


In [7]:
def test(x, y):
    correct = 0
    for xi, yi in zip(x, y):
        pred = predict(xi)
        pred = 1 if pred > 0 else -1
        if pred == yi:
            correct += 1
    return correct / len(x)


test(x, y)

0.923

In [8]:
x, y = load_data('features - 副本.csv')
test(x, y)

0.915505